## Backtrader 基本框架

In [1]:
import pandas as pd
import numpy as np
import backtrader as bt
import tushare as ts
import datetime
import time

### Module 1 获取demo数据
- 基于tushare的daily数据
- 随机模拟每只票的信号 同时

In [86]:
my_tocken = '7d222e720cdaadd5bdb882257fb0a3b33d1485742564a752502e5521'
pro = ts.pro_api(my_tocken)
start_time = "20211017"
end_time = "20211022"

In [3]:
#getting daily price data from tushare
def data_fetcher(start_time, end_time,stocks):
    df_daily_all = pd.DataFrame()
    time_list = pd.date_range(start=start_time, end=end_time)
    for timestamp in time_list:
        date = timestamp.strftime("%Y%m%d")
        print(f"fetching data --date:{date}")
        df_daily = pro.daily(trade_date=date, ts_code=stocks)
        df_daily_all = df_daily_all.append(df_daily)
    return df_daily_all

df_daily_all = data_fetcher(start_time=start_time, end_time=end_time,stocks='000001.SZ,000002.SZ,000004.SZ,000005.SZ')
df_daily_all

fetching data --date:20211017
fetching data --date:20211018
fetching data --date:20211019
fetching data --date:20211020
fetching data --date:20211021
fetching data --date:20211022


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20211018,19.45,19.55,19.10,19.29,19.66,-0.37,-1.8820,729622.50,1408737.777
1,000002.SZ,20211018,21.03,21.08,20.18,20.30,21.09,-0.79,-3.7459,1076032.24,2201113.070
2,000004.SZ,20211018,19.72,19.93,18.91,19.21,19.91,-0.70,-3.5158,37101.84,71289.501
3,000005.SZ,20211018,2.22,2.24,2.19,2.22,2.28,-0.06,-2.6316,204383.73,45358.937
0,000001.SZ,20211019,19.15,19.68,19.15,19.57,19.29,0.28,1.4515,682415.15,1330501.301
1,000002.SZ,20211019,20.52,20.74,20.21,20.30,20.30,0.00,0.0000,637327.40,1299648.695
2,000004.SZ,20211019,19.19,19.40,19.01,19.35,19.21,0.14,0.7288,22380.34,42981.500
3,000005.SZ,20211019,2.22,2.26,2.21,2.22,2.22,0.00,0.0000,84017.00,18735.784
0,000001.SZ,20211020,19.75,19.78,19.19,19.24,19.57,-0.33,-1.6863,662955.55,1283356.219
1,000002.SZ,20211020,20.36,20.57,19.91,20.00,20.30,-0.30,-1.4778,914269.00,1836350.372


In [192]:
cerebro = bt.Cerebro()

#模拟股票信号生成df_signal表，选股逻辑为每个交易日买入信号值最高的两只股票
def get_signal(start_time, end_time,stock_list):
    df_signal = pd.DataFrame()
    time_list = pd.date_range(start=start_time, end=end_time)
    for timestamp in time_list:
        date = timestamp.strftime("%Y%m%d")
        df_signal = df_signal.append(pd.DataFrame([[date]*len(stock_list), stock_list]).T)
    df_signal.columns = ["trade_date","ts_code"]
    df_signal["weight"] = np.random.randn(df_signal.shape[0])
    return df_signal

def get_trade_info(df_signal):
    def apply_func(df):
        df = df.sort_values("weight").iloc[-1]
        return df
    
    trade_info = df_signal.groupby(by="trade_date").apply(apply_func).reset_index(drop=True)
    return trade_info

df_signal = get_signal(start_time=start_time, end_time=end_time, stock_list=['000001.SZ','000002.SZ','000004.SZ','000005.SZ'])
trade_info = get_trade_info(df_signal)

### Module 2 Datafeeds 将数据导入backtrader
- 利用backtrader中的datafeeds模块倒入数据
- 进行格式转换 成为backtrader需要的格式：1.以交易日 'datetime' 为 index   2.列为'open'、'high'、'low'、'close'、'volume'、'openinterest' 字段 

In [193]:
df_daily_all.rename(columns={"vol":"volume"},inplace=True)
df_daily_all['openinterest'] = 0
df_daily_all.index = pd.to_datetime(df_daily_all.trade_date)
trade_info.index = pd.to_datetime(trade_info.trade_date)
# 按股票代码，依次循环传入数据
for stock in df_daily_all['ts_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=df_daily_all.index.unique()) # 获取回测区间内所有交易日
    df = df_daily_all.query(f"ts_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    # 缺失值处理：日期对齐时会使得有些交易日的数据为空，所以需要对缺失数据进行填充
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)
    # 导入数据
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=datetime.datetime.strptime(start_time, "%Y%m%d"), todate=datetime.datetime.strptime(end_time, "%Y%m%d"))
    cerebro.adddata(datafeed, name=stock) # 通过 name 实现数据集与股票的一一对应
    print(f"{stock} Done !")

000001.SZ Done !
000002.SZ Done !
000004.SZ Done !
000005.SZ Done !


### Module 3 backtrader 交易策略
- 通过继承 Strategy 基类，来构建自己的交易策略子类
- demo策略逻辑：每个交易日买入信号值最大的

In [196]:
class TestStrategy(bt.Strategy):
    '''选股策略'''
    def __init__(self):
        self.buy_stock = trade_info # 保留调仓列表
        # 读取调仓日期，即每月的最后一个交易日，回测时，会在这一天下单，然后在下一个交易日，以开盘价买入
        self.trade_dates = pd.to_datetime(self.buy_stock['trade_date'].unique()).tolist()
        self.order_list = [] # 记录以往订单，方便调仓日对未完成订单做处理
        self.buy_stocks_pre = [] # 记录上一期持仓
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已经处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                        'BUY EXECUTED, ref:%.0f，Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                        (order.ref, # 订单编号
                         order.executed.price, # 成交价
                         order.executed.value, # 成交额
                         order.executed.comm, # 佣金
                         order.executed.size, # 成交量
                         order.data._name)) # 股票名称
            else: # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s' %
                            (order.ref,
                             order.executed.price,
                             order.executed.value,
                             order.executed.comm,
                             order.executed.size,
                             order.data._name))
                
    def next(self):
        dt = self.datas[0].datetime.date(0) # 获取当前的回测时间点
        # 如果是调仓日，则进行调仓操作
        if dt in self.trade_dates:
            print("--------------{} 为调仓日----------".format(dt))
            # 在调仓之前，取消之前所下的没成交也未到期的订单
            if len(self.order_list) > 0:
                for od in self.order_list:
                    self.cancel(od) # 如果订单未完成，则撤销订单
                self.order_list = [] #重置订单列表
            # 提取当前调仓日的持仓列表
            buy_stocks_data = self.buy_stock.query(f"trade_date=='{dt.strftime('%Y%m%d')}'")
            long_list = buy_stocks_data['ts_code'].tolist()
            print('long_list', long_list) # 打印持仓列表
            # 对现有持仓中，调仓后不再继续持有的股票进行卖出平仓
            sell_stock = [i for i in self.buy_stocks_pre if i not in long_list]
            print('sell_stock', sell_stock) # 打印平仓列表
            if len(sell_stock) > 0:
                print("-----------对不再持有的股票进行平仓--------------")
                for stock in sell_stock:
                    data = self.getdatabyname(stock)
                    if self.getposition(data).size > 0 :
                        od = self.close(data=data) 
                        self.order_list.append(od) # 记录卖出订单
            # 买入此次调仓的股票：多退少补原则
            print("-----------买入此次调仓期的股票--------------")
            for stock in long_list:
                w = buy_stocks_data.query(f"ts_code=='{stock}'")['weight'].iloc[0] # 提取持仓权重
                print(w)
                data = self.getdatabyname(stock)
                order = self.order_target_percent(data=data, target=w*0.95) # 为减少可用资金不足的情况，留 5% 的现金做备用
                self.order_list.append(order)
       
            self.buy_stocks_pre = long_list # 保存此次调仓的股票列表
        

### Module 4 cerebro 启动引擎 & 获取结果
- 通过brocker设置初始资金，佣金，滑点
- 通过 analyzers 策略分析模块和 observers 观测器模块提前配置好要返回的回测结果
- 启动引擎 开始回测

In [197]:

# 将编写的策略添加给大脑，别忘了 ！
cerebro.addstrategy(TestStrategy)

# 初始资金 100,000,000 
cerebro.broker.setcash(100000000.0) 
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003) 
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤

result = cerebro.run()

--------------2021-10-18 为调仓日----------
trade_date==20211810
long_list ['000001.SZ']
sell_stock []
-----------买入此次调仓期的股票--------------
0.0637392704318646
--------------2021-10-18 为调仓日----------
long_list ['000001.SZ']
sell_stock []
-----------买入此次调仓期的股票--------------
0.0637392704318646
2021-10-19, BUY EXECUTED, ref:10，Price: 19.15, Cost: 6011881.88, Comm 1803.56, Size: 313905.00, Stock: 000001.SZ
--------------2021-10-19 为调仓日----------
trade_date==20211910
long_list ['000005.SZ']
sell_stock ['000001.SZ']
-----------对不再持有的股票进行平仓--------------
-----------买入此次调仓期的股票--------------
1.0110672126410911
2021-10-19, BUY EXECUTED, ref:11，Price: 19.15, Cost: 6011881.88, Comm 1803.56, Size: 313905.00, Stock: 000001.SZ
--------------2021-10-19 为调仓日----------
long_list ['000005.SZ']
sell_stock ['000001.SZ']
-----------对不再持有的股票进行平仓--------------
-----------买入此次调仓期的股票--------------
1.0110672126410911
2021-10-20, SELL EXECUTED, ref:12, Price: 19.75, Cost: 12023763.76, Comm 3719.40, Size: -627810.00, St